In [3]:
import numpy as np
import gc;from datetime import datetime;import pandas as pd
import warnings;warnings.simplefilter(action='ignore', category=Warning)

In [4]:
file_path = 'C:/temp/clean_zzz.csv'
zzz = pd.read_csv(file_path)

In [5]:
# creating possible time covariates for model 
start_time = datetime.strptime(zzz['timestamp'].iloc[0], '%Y-%m-%dT%H:%M:%S%z')
zzz['seconds_since_start'] = zzz['timestamp'].apply(lambda x: (datetime.strptime(x, '%Y-%m-%dT%H:%M:%S%z') - start_time).seconds)
zzz = zzz.drop(columns=['timestamp', 'step'])

In [8]:
# sorte by time series_id & seconds_since_start
zzz_timesort = zzz.sort_values(by=['series_id', 'seconds_since_start'])

In [9]:
# Standardize feats: time,anglez & enmo 
zzz_timesort['anglez_std'] = (zzz_timesort['anglez'] - zzz_timesort['anglez'].mean()) / zzz_timesort['anglez'].std()
zzz_timesort['enmo_std'] = (zzz_timesort['enmo'] - zzz_timesort['enmo'].mean()) / zzz_timesort['enmo'].std()
zzz_timesort['sss_std'] = (zzz_timesort['seconds_since_start'] - zzz_timesort['seconds_since_start'].mean()) / zzz_timesort['seconds_since_start'].std()

# #set for Linear Model
feats = [col for col in zzz_timesort.columns if 'std' in col]
X = zzz_timesort[feats]
y = zzz_timesort['awake']

# split the data (79-21)
train_size = int(0.79 * len(zzz_timesort))
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

In [14]:
X_train.describe()

,anglez_std,enmo_std,sss_std
count,1.040079e+07,1.040079e+07,1.040079e+07
mean,6.557426e-03,-3.195460e-03,-1.461620e-02
std,9.848785e-01,9.881474e-01,9.964528e-01
min,-2.079821e+00,-3.873102e-01,-1.738467e+00
25%,-7.478647e-01,-3.767575e-01,-8.727772e-01
50%,-1.709713e-02,-2.430901e-01,-2.197521e-02
75%,6.672965e-01,-2.939801e-02,8.392883e-01
max,2.552685e+00,6.131166e+01,1.737771e+00


In [15]:
file_path1 = 'C:/temp/X1.csv'
file_path2 = 'C:/temp/y1.csv'
file_path3 = 'C:/temp/X2.csv'
file_path4 = 'C:/temp/y2.csv'
X_train.to_csv(file_path1, index=False)
y_train.to_csv(file_path2, index=False)
X_test.to_csv(file_path3, index=False)
y_test.to_csv(file_path4, index=False)

In [16]:
X_train = pd.read_csv(file_path1)
y_train = pd.read_csv(file_path2)
X_test = pd.read_csv(file_path3)
y_test = pd.read_csv(file_path4)